In [16]:
import numpy as np
from scipy.optimize import minimize
MJ_TO_KCAL = 239

# Parameters
rho_G = 17.6  # MJ/kg, energy density of glycogen
rho_F = 39.5  # MJ/kg, energy density of fat
rho_L = 7.6   # MJ/kg, energy density of lean tissue
gamma_F = 13  # kJ/kg/day, coefficient for fat
gamma_L = 92  # kJ/kg/day, coefficient for lean mass
eta_F = 750   # kJ/kg, efficiency associated with fat
eta_L = 960   # kJ/kg, efficiency associated with lean tissue
beta_TEF = 0.1  # Thermic effect of feeding
beta_AT = 0.14  # Adaptive thermogenesis parameter
tau_AT = 14  # days, timescale for adaptive thermogenesis dynamics
C = 10.4 * rho_L / rho_F  # Constant for energy partitioning function


In [17]:
def calculate_initial_fat(BW, H, age, sex):
    """
    Calculate initial body fat using regression equations.
    BW: Body weight in kg
    H: Height in meters
    age: Age in years
    sex: 'male' or 'female'
    """
    if sex == 'male':
        return 0.1 * BW + 0.23 * H - 0.54 * age - 13.2
    else:  # female
        return 0.07 * BW + 0.25 * H - 0.54 * age - 6.3

def total_energy_expenditure(EI, F, L):
    """
    Calculate total energy expenditure rate (EE).
    EI: Energy intake rate
    F: Body fat
    L: Lean tissue mass
    """
    p = C / (C + F)
    EE = EI - (rho_F * p + rho_L * (1 - p)) * (EI - gamma_F * F - gamma_L * L) / (eta_F * p + eta_L * (1 - p))
    return EE

def body_weight_change(EI, F, L, delta_EI, delta_activity):
    """
    Calculate body weight change.
    EI: Energy intake rate
    F: Body fat
    L: Lean tissue mass
    delta_EI: Change in energy intake
    delta_activity: Change in physical activity
    """
    EE = total_energy_expenditure(EI, F, L)
    delta_EE = delta_EI + delta_activity * EE
    delta_BW = delta_EE / (gamma_L - gamma_F)
    return delta_BW


In [18]:
# Example usage
BW_initial = 70  # initial body weight in kg
H = 1.80  # height in meters
age = 23  # age in years
sex = 'male'  # sex
EI = 10  # energy intake rate in MJ/day (example value)
delta_EI = -1  # change in energy intake in MJ/day (example value) represents the change in energy intake that the user plans to make to achieve their weight goal. This could be a reduction or increase in caloric intake compared to their current diet. You might ask the user how many calories they plan to add or subtract from their daily diet.
delta_activity = 0.1  # change in physical activity level (example value)

# Calculate initial body fat and lean tissue mass
F_initial = calculate_initial_fat(BW_initial, H, age, sex)
L_initial = BW_initial - F_initial

# Calculate body weight change
delta_BW = body_weight_change(EI, F_initial, L_initial, delta_EI, delta_activity)
print(f"Predicted body weight change: {delta_BW:.2f} kg")


Predicted body weight change: 0.04 kg


In [27]:
def calculate_updated_fat(BW, F_current, delta_BW):
    """
    Calculate updated body fat based on change in body weight.
    BW: current body weight
    F_current: current body fat
    delta_BW: change in body weight
    """
    FAT_LOSS_PERCENTAGE = 0.75  # Placeholder value, replace with actual percentage
    FAT_GAIN_PERCENTAGE = 0.75  # Placeholder value, replace with actual percentage
    
    if delta_BW < 0:  # Weight loss
        delta_F = abs(delta_BW) * FAT_LOSS_PERCENTAGE
        F_updated = max(F_current - delta_F, 0)  # Ensure body fat doesn't go negative
    else:  # Weight gain
        delta_F = delta_BW * FAT_GAIN_PERCENTAGE
        F_updated = F_current + delta_F
    
    return F_updated


In [22]:
def simulate_weight_change(EI, F_initial, L_initial, days):
    """
    Simulate weight change over a specified number of days based on energy intake.
    """
    F = F_initial
    L = L_initial
    BW = F + L
    for day in range(days):
        delta_BW = body_weight_change(EI, F, L, delta_EI=0, delta_activity=0)
        BW += delta_BW
        F = calculate_updated_fat(BW, F, delta_BW)  # You need to define this function based on your model
        L = BW - F
    return BW


In [28]:
def objective_function(EI, goal_weight, F_initial, L_initial, days):
    """
    Objective function to minimize.
    The goal is to find the EI that results in the goal weight within the specified time frame.
    """
    final_weight = simulate_weight_change(EI, F_initial, L_initial, days)
    return (final_weight - goal_weight)**2

def find_energy_intake(goal_weight, days, F_initial, L_initial, initial_guess=10):
    """
    Find the energy intake that leads to the goal weight within the specified time frame.
    """
    result = minimize(objective_function, initial_guess, args=(goal_weight, F_initial, L_initial, days))
    if result.success:
        return result.x[0]  # Return the optimized energy intake
    else:
        raise ValueError("Optimization failed to find the energy intake for the goal weight")

# User inputs
goal_weight = float(input("Enter your goal weight in kg: "))
days_to_goal = int(input("Enter the number of days until you would like to reach your goal: "))

# Example usage
EI_optimized = find_energy_intake(goal_weight, days_to_goal, F_initial, L_initial)
print(f"To reach a goal weight of {goal_weight} kg in {days_to_goal} days, consume {EI_optimized * MJ_TO_KCAL:.2f} kcal/day")


To reach a goal weight of 80.0 kg in 30 days, consume 60958.76 kcal/day
